In [1]:
import pandas as pd
import re

In [2]:
pd.options.display.max_columns = 99

In [3]:
df = pd.read_csv('./data-example.csv')
column_names = list(df)

In [5]:
list(df)

['Unique Response Number',
 'Introduction',
 '1. I agree to share the information provided in this survey. (Note we will not share your name, role or email address).',
 '2. I will provide the answers that reflect the best of my knowledge and facts available to me.',
 '3. I have received adequate information about the survey and about my ethical rights as a participant, including my right to withdraw. I consent to take part in this research',
 '4. Your name',
 '4.a. Your role',
 '4.b. Contact details (preferably e-mail address)',
 '4.c. Second/alternative contact detail.',
 "4.d. Your organisation (If you are not affiliated with an organisation, please write 'not applicable').",
 '5. Are you reporting as an individual or on behalf of an organisation? (Several reports from the same organisation or individual are welcome)',
 '5.a. If you selected Other, please specify:',
 '5.b. On behalf of which organisation are you reporting here? (e.g. UNEP, FAO or name your specific private/public/thi

In [414]:
def word_count(strings):
    words = strings.split(' ')
    return len(words)

In [415]:
def category_count(words):
    counts = dict()
    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

In [416]:
def get_column_info(df, data):
    col_id = None
    col_name = data
    if '.' in data:
        column = data.split(' ')
        col_id = column[0].replace('.','').upper()
        col_name = ' '.join(column[1:-1])
    col_type = str(df[data].dtypes)
    col_category = []
    if col_type == 'object':
        categories = df[data].unique().tolist()
        for value in categories:
            if type(value) == str:
                if '://' in value or 'UTC' in value or 'www' in value:
                    value = ''
                value = value.replace(' / ','/')
            if type(value) == float:
                pass
            elif '|' in value and type(value) == str:
                categories = value.split('|')
                for cat in categories:
                    col_category.append(cat)
            elif value == '' and value == None:
                pass
            else:
                words = word_count(str(value))
                if words > 30:
                    pass
                else:
                    col_category.append(value)
    if 'link' in data:
        col_category = []
    col_category = '#'.join(col_category)
    col_category = col_category.replace('#',',')
    col_category = col_category.replace('/',',')
    col_category = col_category.replace('),,',')#')
    col_category = col_category.replace(').',')#')
    col_category = col_category.replace('),',')#')
    if '(' in col_category:
        col_category = col_category.split('#')
    else:
        col_category = col_category.split(',')
    category = []
    for cat in col_category:
        if cat == '':
            pass
        else:
            category.append(cat.strip())
    if col_name == 'Unique Response Number':
        category = []
    category = category_count(category)
    col_category = []
    for v in category:
        col_category = col_category
        col_category.append({
            'name':v.strip(','),
            'total': category[v]
        })
    if len(col_category) == 0:
        col_category = None
    return {
        'id': col_id,
        'name': col_name,
        'type': col_type,
        'categories': col_category
    }

In [417]:
col_info = [get_column_info(df, x) for x in column_names]

In [418]:
pd.DataFrame(col_info).to_json('data-info.json', orient='records')

In [419]:
pd.DataFrame(col_info).to_dict('records')

[{'id': None,
  'name': 'Unique Response Number',
  'type': 'object',
  'categories': None},
 {'id': None, 'name': 'Introduction', 'type': 'float64', 'categories': None},
 {'id': '1',
  'name': 'I agree to share the information provided in this survey. (Note we will not share your name, role or email',
  'type': 'float64',
  'categories': None},
 {'id': '2',
  'name': 'I will provide the answers that reflect the best of my knowledge and facts available to',
  'type': 'float64',
  'categories': None},
 {'id': '3',
  'name': 'I have received adequate information about the survey and about my ethical rights as a participant, including my right to withdraw. I consent to take part in this',
  'type': 'float64',
  'categories': None},
 {'id': '4', 'name': 'Your', 'type': 'float64', 'categories': None},
 {'id': '4A', 'name': 'Your', 'type': 'float64', 'categories': None},
 {'id': '4B',
  'name': 'Contact details (preferably e-mail',
  'type': 'float64',
  'categories': None},
 {'id': '4C',
  